In [1]:
%%capture
!pip install stanza

In [2]:
%%capture
import stanza

stanza.download('es')
nlp = stanza.Pipeline('es')

In [109]:
file1 = open('dic-judeo-español.txt', 'r', encoding="utf-8")
Lines = file1.readlines()
dic=[]
for l in Lines:
  p = {"src":l.split(";")[0],"target":l.split(";")[1]}
  dic.append(p)

In [110]:
def judeo_parse(s):
  if s.find("h") != -1:
    indices = [i for i, x in enumerate(s) if x == "h"]
    flag_h = 0
    for h in indices:
      string_list = list(s)
      if h != 0:
        if s[h-2] != "c":
          string_list[h] = ""
      else:
        string_list[0] = ""
    s = "".join(string_list)
  if s.find("y") != -1:
    if len(s) == 1:
      s = s.replace("y","i")
  if s.find("ca") != -1:
    s = s.replace("ca","ka")
  if s.find("co") != -1:
    s = s.replace("co","ko")
  if s.find("cu") != -1:
    s = s.replace("cu","ku")
  if s.find("ñ") != -1:
    s = s.replace("ñ","ny")
  if s.find("qu") != -1:
    s = s.replace("qu","k")
  if s.find("rd") != -1:
    s = s.replace("rd","dr")
  if s.find("cr") != -1:
    s = s.replace("cr","kr")
  if s.find("ll") != -1:
    s = s.replace("ll","y")
  if s.find("g") != -1:
    s = s.replace("g","j")
  return s

In [111]:
def complete_word(mayor,menor):
  for l in range(len(mayor)-len(menor)):
    menor = menor+" "
  return menor

def get_suffix(a,b):
  flag = 0
  mayor = ""
  menor = ""

  if len(a) > flag:
    mayor = a
    menor = b
    flag = len(a)
  if len(b) > flag:
    mayor = b
    menor = a
  menor = complete_word(mayor,menor)

  f = []
  for x, y in zip(mayor,menor):
    if x == y:
      f.append(x)

  h = ''.join([str(elem) for elem in f])

  for l in range(len(a)-len(h)):
    h = h+" "

  p = []
  for x, y in zip(a,h):
    if x != y:
      p.append(x)

  p = ''.join([str(elem) for elem in p])
  
  return p

In [112]:
def conj_verb(verbo_esp,verbo_ladino):
  verbo_phrase = verbo_esp.text
  print(verbo_esp)
  print("Verbo Español: ",verbo_phrase)
  print("Verbo Ladino: ",verbo_ladino)
  verbo_inf = verbo_esp.lemma
  if "ar" in verbo_phrase[-2] or "er" in verbo_phrase[-2] or "ir" in verbo_phrase[-2] or "VerbForm=Inf" in verbo_esp.feats:
    verbo_ladino = verbo_ladino
  #if "ar" in verbo_ladino[len(verbo_ladino)-2:len(verbo_ladino)]:
  #  verbo_ladino = ''.join(verbo_ladino.rsplit('ar',1))
  #if "er" in verbo_ladino[len(verbo_ladino)-2:len(verbo_ladino)]:
  #  verbo_ladino = ''.join(verbo_ladino.rsplit('er',1))
  #if "ir" in verbo_ladino[len(verbo_ladino)-2:len(verbo_ladino)]:
  #  verbo_ladino = ''.join(verbo_ladino.rsplit('ir',1))
  elif "ando" in verbo_phrase:
    verbo_ladino = verbo_ladino[:len(verbo_ladino)-2]+"ando"
  elif "iendo" in verbo_phrase:
    verbo_ladino = verbo_ladino+"iendo"
  elif "ado" in verbo_phrase:
    verbo_ladino = verbo_ladino+"ado"
  elif "ido" in verbo_phrase:
    verbo_ladino = verbo_ladino+"ido"
  else:
    suff = get_suffix(verbo_phrase.lower(),verbo_inf)
    print("Verbo Español en Infinitivo: ",verbo_inf)
    print("Sufijo Español: ",suff)
    if "Number=Sing|Person=1|Tense=Past" in verbo_esp.feats or "Number=Sing|Person=3|Tense=Past" in verbo_esp.feats or "Number=Sing|Person=1|Tense=Pres" in verbo_esp.feats:
      root_ladino = verbo_ladino[:len(verbo_ladino)-2]
      if root_ladino[-1] == suff[:1]:
        verbo_ladino = verbo_ladino[:len(verbo_ladino)-3]+suff
      else:
        verbo_ladino = verbo_ladino[:len(verbo_ladino)-2]+suff
    if "Number=Sing|Person=3|Tense=Fut" in verbo_esp.feats or "Number=Sing|Person=1|Tense=Fut" in verbo_esp.feats or "Number=Plur|Person=3|Tense=Fut" in verbo_esp.feats or "Number=Plur|Person=3|Tense=Past" in verbo_esp.feats or "Number=Plur|Person=1|Tense=Fut" in verbo_esp.feats:
      verbo_ladino = verbo_ladino+suff
    if "Number=Sing|Person=2|Tense=Pres" in verbo_esp.feats or "Number=Plur|Person=3|Tense=Pres" in verbo_esp.feats or "Number=Plur|Person=1|Tense=Pres" in verbo_esp.feats:
      verbo_ladino = verbo_ladino[:len(verbo_ladino)-1]+suff
  print("Verbo Ladino Final: ",verbo_ladino)
  return verbo_ladino

In [113]:
def conj_aux(verbo_esp,verbo_ladino):
  verbo_phrase = verbo_esp.text
  print(verbo_esp)
  print("Verbo Español: ",verbo_phrase)
  print("Verbo Ladino: ",verbo_ladino)
  verbo_inf = verbo_esp.lemma
  suff = get_suffix(verbo_phrase.lower(),verbo_inf)
  print("Verbo Español en Infinitivo: ",verbo_inf)
  print("Sufijo Español: ",suff)
  if "Number=Sing|Person=3|Tense=Imp" in verbo_esp.feats:
    verbo_ladino = verbo_ladino[:len(verbo_ladino)-1]+suff
  elif "|Number=Sing|Person=3|Tense=Pres|" in verbo_esp.feats or "|Number=Sing|Person=1|Tense=Pres|" in verbo_esp.feats or "|Number=Plur|Person=3|Tense=Pres| in verbo_esp.feats":
    verbo_ladino = verbo_ladino[:len(verbo_ladino)-2]+suff
  print("Verbo Ladino Final: ",verbo_ladino)
  return verbo_ladino

In [114]:
def conj_adj_sustantivo(word_esp,word_ladino):
  print(word_esp)
  if str(word_esp.feats) != "None":
    if "Sing" in word_esp.feats:
      return word_ladino
    elif "Plur" in word_esp.feats:
      if (word_esp.text[-2:].find("a") != -1 and word_ladino[-2:].find("a") != -1) or (word_esp.text[-2:].find("o") != -1 and word_ladino[-2:].find("o") != -1) :
        print("entro")
        suff_word_esp = word_esp.text[len(word_esp.text)-2:]
        root_word_lad = word_ladino[:len(word_ladino)-1]
        word_ladino = root_word_lad+suff_word_esp
        return word_ladino
      elif word_esp.text[-1].find("s") != -1:
        if word_esp.text[-3:] == "ses":
          word_ladino = word_ladino+"es"
        else:
          word_ladino = word_ladino+"s"
        return word_ladino
      else:
        return word_ladino+"s"
    else:
      return word_ladino
  return word_ladino

In [115]:
def translate(phrase):
  doc = nlp(phrase)
  jud_phrase = ""
  w = ""
  for sent in doc.sentences:
    for word in sent.words:
      flag1 = 0
      flag2 = 0
      español_word = word.text
      print(español_word)
      mixed_case = not español_word.islower() and not español_word.isupper()
      if mixed_case == True:
        flag2 = 1
      for d in dic:
        if word.text.lower() == d["src"]:
          print("Misma Palabra: ",d["src"])
          ladino_word = d["target"].replace(" ","").replace("\n","")
          if word.lemma == ladino_word or word.text.lower() == ladino_word:
            w = español_word
          else:
            w = ladino_word
          flag1 = 1
      if flag1 == 0:
        for d in dic:
          if word.lemma == d["src"]:
            print("Misma infinitivo: ",d["src"])
            ladino_word = d["target"].replace(" ","").replace("\n","")
            if word.lemma == ladino_word or word.text.lower() == ladino_word:
              w = español_word
            if word.upos == "VERB":
              print("VERB")
              w = conj_verb(word,ladino_word)
              flag2 = 0
            elif word.upos == "NOUN" or word.upos == "ADJ":
              print("NOUN")
              w = conj_adj_sustantivo(word,ladino_word)
            elif word.upos == "AUX":
              print("AUX")
              w = conj_aux(word,ladino_word)
            elif word.upos == "DET":
              w = español_word
            else:
              print(word)
              print("OTRA COSA")
              w = ladino_word
            flag1 = 1
      if flag1 == 0:
        if word.upos == "PROPN" or word.upos == "DET":
          w = word.text
        else:
          w = judeo_parse(español_word)
      if flag2 == 1:
        jud_phrase += w.replace("\n","").capitalize() + " "
      else:
        jud_phrase += w.replace("\n","") + " "
      print("\n")
  return jud_phrase.replace(" .",".").replace(" ?","?").replace(" !","!").replace(" ,",",").replace("de el","del").replace('" ','"').replace(" ;",";").replace("¿ ","¿").strip()

In [116]:
phrase="Los estudiantes están estudiando."
translate(phrase)

Los
Misma infinitivo:  el


estudiantes
Misma infinitivo:  estudiante
NOUN
{
  "id": 2,
  "text": "estudiantes",
  "lemma": "estudiante",
  "upos": "NOUN",
  "feats": "Number=Plur",
  "head": 4,
  "deprel": "nsubj",
  "start_char": 4,
  "end_char": 15
}


están
Misma infinitivo:  estar
AUX
{
  "id": 3,
  "text": "están",
  "lemma": "estar",
  "upos": "AUX",
  "feats": "Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin",
  "head": 4,
  "deprel": "aux",
  "start_char": 16,
  "end_char": 21
}
Verbo Español:  están
Verbo Ladino:  estar
Verbo Español en Infinitivo:  estar
Sufijo Español:  án
Verbo Ladino Final:  están


estudiando
Misma infinitivo:  estudiar
VERB
{
  "id": 4,
  "text": "estudiando",
  "lemma": "estudiar",
  "upos": "VERB",
  "feats": "VerbForm=Ger",
  "head": 0,
  "deprel": "root",
  "start_char": 22,
  "end_char": 32
}
Verbo Español:  estudiando
Verbo Ladino:  estudiar
Verbo Ladino Final:  estudiando


.




'Los elevos están estudiando.'

Testing

In [117]:
import pandas as pd

df = pd.read_excel("una-fraze-al-dia_lad-tur-eng-spa.xlsx")

esp=[]
lad_correct = []
lad_trans = []

for a in df.index:
  print(df["Español"][a])
  esp.append(df["Español"][a])
  lad_correct.append(df["Ladino"][a])
  lad_trans.append(translate(df["Español"][a]))

p = {'Español':esp,'Ladino':lad_correct,'Ladino_Trans':lad_trans}

df_1 = pd.DataFrame(p)
df_1.head()

df_1.to_excel("translate_esp_ladino.xlsx")

Estoy aprendiendo el judeoespañol.
Estoy
Misma infinitivo:  estar
AUX
{
  "id": 1,
  "text": "Estoy",
  "lemma": "estar",
  "upos": "AUX",
  "feats": "Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin",
  "head": 2,
  "deprel": "aux",
  "start_char": 0,
  "end_char": 5
}
Verbo Español:  Estoy
Verbo Ladino:  estar
Verbo Español en Infinitivo:  estar
Sufijo Español:  oy
Verbo Ladino Final:  estoy


aprendiendo
Misma Palabra:  aprendiendo


el
Misma Palabra:  el


judeoespañol
Misma Palabra:  judeoespañol


.


Me levanto a las siete cada día.
Me


levanto
Misma Palabra:  levanto


a


las
Misma infinitivo:  el


siete
Misma Palabra:  siete


cada
Misma Palabra:  cada


día
Misma Palabra:  día
Misma Palabra:  día


.


Me gusta leer.
Me


gusta
Misma Palabra:  gusta
Misma Palabra:  gusta


leer
Misma Palabra:  leer


.


Bebo café turco después del almuerzo.


/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


Se han truncado las últimas 5000 líneas del flujo de salida.
Me


cansé


mucho
Misma Palabra:  mucho


.


Cuando eramos pequeños, no teníamos televisión en casa.
Cuando
Misma Palabra:  cuando


eramos


pequeños
Misma infinitivo:  pequeño
NOUN
{
  "id": 3,
  "text": "pequeños",
  "lemma": "pequeño",
  "upos": "ADJ",
  "feats": "Gender=Masc|Number=Plur",
  "head": 6,
  "deprel": "advcl",
  "start_char": 14,
  "end_char": 22
}
entro


,


no


teníamos
Misma infinitivo:  tener
VERB
{
  "id": 6,
  "text": "teníamos",
  "lemma": "tener",
  "upos": "VERB",
  "feats": "Mood=Ind|Number=Plur|Person=1|Tense=Imp|VerbForm=Fin",
  "head": 0,
  "deprel": "root",
  "start_char": 27,
  "end_char": 35
}
Verbo Español:  teníamos
Verbo Ladino:  tener
Verbo Español en Infinitivo:  tener
Sufijo Español:  íamos
Verbo Ladino Final:  tener


televisión
Misma Palabra:  televisión


en


casa
Misma Palabra:  casa
Misma Palabra:  casa


.


El martes por la noche terminó el año 2019 y empezó el año 2020.
El
M

In [ ]:
file1 = open('dic-judeo-español.txt', 'r', encoding="utf-8")
Lines = file1.readlines()
dic=[]
for l in Lines:
  word_esp = l.split(";")[0]
  word_ladino = l.split(";")[1]
  if len(word_esp.split(" ")) == 1 or len(word_esp.split(",")) == 1: 
    doc = nlp(word_esp)
    for sent in doc.sentences:
      for word in sent.words:
        if word.upos == "NOUN" and str(word.feats) != "None":
          if "Sing" in word.feats and "Masc" in word.feats:
            if word_esp[len(word_esp)-1:] == "o":
              root_word_esp = word_esp[:len(word_esp)-1]
              root_word_lad = word_ladino[:len(word_ladino)-2]
              word_esp = root_word_esp+"a"
              word_ladino = root_word_lad+"a"
              p = word_esp+";"+word_ladino
              dic.append(p)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


Using a new POS-Tagging for Spanish

In [ ]:
!pip install nltk

In [ ]:
import nltk
tagged_sentences = nltk.corpus.conll2002.tagged_sents('esp.train')

In [ ]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:3],
        'prefix-3': sentence[index][:4],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-3:],
        'suffix-3': sentence[index][-4:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

In [ ]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [ ]:
cutoff = int(.75 * len(tagged_sentences))
training_sentences = tagged_sentences[:cutoff]
test_sentences = tagged_sentences[cutoff:]

print(len(training_sentences))
print(len(test_sentences))

def transform_to_dataset(tagged_sentences):
    X, y = [], []
    
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged),index))
            y.append(tagged[index][1])
    return X, y

X, y = transform_to_dataset(training_sentences)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])

clf.fit(X[:20000], y[:20000])

print('Training completed')

In [ ]:
def pos_tag(sentence):
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    return list(zip(sentence, tags))

In [ ]:
from nltk import word_tokenize
print (pos_tag(word_tokenize('El canciller de Alemania llegó el lunes a Rusia')))

In [3]:
file1 = open('dic-judeo-español.txt', 'r', encoding="utf-8")
Lines = file1.readlines()
dic=[]
for l in Lines:

  if l.find(",") == -1:
    dic.append(l)

In [6]:
dic = sorted(set(dic))
for f in dic:
  print(f.replace("\n",""))

a donde;ande
a escondidas;a las eskondidas
a la vez;en mizmo tiempo
a menudo;akoruto
a partir de;a partir de
a pie;a pie
a punto de;sovre el punto
a rajatabla;a la letra
a sabiendas;todo en saviendo
a ver;veremos
abalanzar;ekilibrar
abandonar;abandonar
abarcar;abrasar
abogada;avokata
abogado;avokato
abrazar;abrasar
abriga;pardesa
abrigarse;abrigarse
abrigo;pardesu
abril;avril
abringandonos;abrigandomos
abrir;avrir
absoluto;absoluto
abuela;nona
abuelo;nono
aburrir;enfasyar
acabar;akavar
acabar;eskapar
acacia;akasia
acariciar;karesar
accidente;aksidente
acción;aksion
aceite;azete
acenta;aksaa
acento;aksan
acentuación;aksantuasion
acentuar;aksantuar
aceptar;akseptar
acercar;aserkar
acercarse;aserkarse
achaque;desrepozo
acomodar;akomodar
acompaña;akompanya
acompañar;akompanyar
aconsejar;konsejar
acordar;akordar
acordarse;akordarse
acostarse;echarse
acostumbrado;alishtereado
acostumbrar;akostumbrar
acta;akta
acto;akto
actor;aktor
actriz;aktrisa
actual;aktual
actuar;aktuar
acuerda;akorda
acu

In [ ]:
df_2 = pd.DataFrame(dic)
df_2.to_excel("dic.xlsx")